In [1]:
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.float_format = '{:.3f}'.format

In [2]:
# Read the right files to create our DB

link_bas = r"C:\Users\User\Wild_Code\Projects\Project_2\DB\title.basics.tsv.gz"
title_bas = pd.read_csv(link_bas, sep = "\t", low_memory = False, na_values = r"\N")

link_akas = r"C:\Users\User\Wild_Code\Projects\Project_2\DB\title.akas.tsv.gz"
title_akas = pd.read_csv(link_akas, sep = "\t", low_memory = False, na_values = r"\N")

In [3]:
title_bas[title_bas["titleType"] == "movie"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 677353 entries, 8 to 10692994
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          677353 non-null  object 
 1   titleType       677353 non-null  object 
 2   primaryTitle    677351 non-null  object 
 3   originalTitle   677351 non-null  object 
 4   isAdult         677353 non-null  float64
 5   startYear       579409 non-null  float64
 6   endYear         0 non-null       float64
 7   runtimeMinutes  426122 non-null  object 
 8   genres          602625 non-null  object 
dtypes: float64(3), object(6)
memory usage: 51.7+ MB


In [4]:
link_rate = r"C:\Users\User\Wild_Code\Projects\Project_2\DB\title.ratings.tsv.gz"
title_rate = pd.read_csv(link_rate, sep = "\t", na_values = r"\N")

In [5]:
TMdb = pd.read_csv(r"C:\Users\User\Wild_Code\Projects\Project_2\DB\tmdb_full.csv", low_memory = False)

In [6]:
# Get the list of existing type in title_basics
title_bas["titleType"].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [7]:
# Creation of the DB with only movies without genre Adult
df_movie = title_bas[(~title_bas["genres"].str.contains("Adult", na = False)) & (title_bas["titleType"].str.contains("movie", na = False))]

In [8]:
len(df_movie["tconst"])

668387

In [9]:
title_akas.sample(5)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
24339505,tt20853786,4,Épisode #31.7466,FR,fr,NaN,NaN,0
12919221,tt13360556,3,एपिसोड #5.132,IN,hi,NaN,NaN,0
5806482,tt10683688,7,エピソード #1.1217,JP,ja,NaN,NaN,0
43929541,tt8099886,6,Folge #1.101,DE,de,NaN,NaN,0
15128616,tt14234556,4,Épisode #1.177,FR,fr,NaN,NaN,0


In [10]:
# Creation of the list of traduced title
fr_akas = title_akas[["titleId","title", "types", "region"]][title_akas["region"].str.contains("FR", na = False)]

In [11]:
fr_akas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4696869 entries, 12 to 47953103
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   titleId  object
 1   title    object
 2   types    object
 3   region   object
dtypes: object(4)
memory usage: 179.2+ MB


In [12]:
fr_akas["types"].value_counts()

types
imdbDisplay    158705
alternative      6042
working          2220
tv               1804
dvd              1528
video            1503
festival          945
Name: count, dtype: int64

In [13]:
# Add the title type on akas to filter juste the movies
fr_akas = fr_akas.merge(title_bas["titleType"], how = "left", left_on = fr_akas["titleId"], right_on = title_bas["tconst"])

In [14]:
fr_akas = fr_akas[fr_akas["titleType"] == "movie"]

In [15]:
fr_akas

,key_0,titleId,title,types,region,titleType
366,tt0000591,tt0000591,L'enfant prodigue,imdbDisplay,FR,movie
392,tt0000886,tt0000886,Hamlet,imdbDisplay,FR,movie
410,tt0001101,tt0001101,La Clémence d'Abraham Lincoln,imdbDisplay,FR,movie
411,tt0001109,tt0001109,L'aluminite,imdbDisplay,FR,movie
413,tt0001122,tt0001122,L'auberge rouge,imdbDisplay,FR,movie
...,...,...,...,...,...,...
4696279,tt9913936,tt9913936,Le paradis de Diego,imdbDisplay,FR,movie
4696364,tt9914164,tt9914164,Une vie de château,imdbDisplay,FR,movie
4696701,tt9915790,tt9915790,Bobbyr Bondhura,imdbDisplay,FR,movie
4696752,tt9916162,tt9916162,Making of 'La Virgen de los sicarios',imdbDisplay,FR,movie


In [16]:
fr_akas["types"].value_counts()

types
imdbDisplay    64859
alternative     3239
working         1678
tv              1402
video           1304
dvd              970
festival         736
Name: count, dtype: int64

In [17]:
fr_akas[fr_akas.duplicated(subset = "titleId", keep = False)]

,key_0,titleId,title,types,region,titleType
532,tt0002423,tt0002423,La Du Barry,NaN,FR,movie
533,tt0002423,tt0002423,Madame du Barry,alternative,FR,movie
534,tt0002423,tt0002423,Passion,imdbDisplay,FR,movie
566,tt0002844,tt0002844,The Phantom Crook,imdbDisplay,FR,movie
567,tt0002844,tt0002844,Fantômas,NaN,FR,movie
...,...,...,...,...,...,...
4692232,tt9896916,tt9896916,Le Voyage du Pèlerin,imdbDisplay,FR,movie
4693754,tt9904530,tt9904530,Scream Returns,imdbDisplay,FR,movie
4693755,tt9904530,tt9904530,Scream Returns - Fan Film Spin-Off,NaN,FR,movie
4694579,tt9907782,tt9907782,The Cursed,alternative,FR,movie


In [18]:
dupli_akas = fr_akas[fr_akas.duplicated(subset = "titleId", keep = False)]

In [19]:
len(dupli_akas["titleId"].unique())

8190

In [20]:
len(dupli_akas["titleId"][dupli_akas["types"] == "imdbDisplay"].unique())

6613

In [21]:
dupli_akas["types"].value_counts()

types
imdbDisplay    7023
alternative    2646
working        1572
video           887
tv              621
dvd             519
festival        224
Name: count, dtype: int64

In [22]:
dupli_akas["types"] = dupli_akas["types"].apply(lambda x: "aaimdbDisplay" if x == "imdbDisplay" else x)

C:\Users\User\AppData\Local\Temp\ipykernel_28880\3779870368.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupli_akas["types"] = dupli_akas["types"].apply(lambda x: "aaimdbDisplay" if x == "imdbDisplay" else x)


In [23]:
dupli_akas.sort_values(by = "types", inplace = True)


C:\Users\User\AppData\Local\Temp\ipykernel_28880\3440819965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupli_akas.sort_values(by = "types", inplace = True)


In [46]:
# Removing the duplicated lines (same movie with different fr akas)
dupli_akas.drop_duplicates(subset = ["titleId"], inplace = True)

C:\Users\User\AppData\Local\Temp\ipykernel_28880\1851943322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupli_akas.drop_duplicates(subset = ["titleId"], inplace = True)


In [47]:
# Removing all duplicated lines (same movie with different fr akas)
fr_akas.drop_duplicates(subset = ["titleId"], keep = False, inplace = True)

In [48]:
# Add the unique title from the duplicated list 
fr_akas = pd.concat([fr_akas, dupli_akas])

In [49]:
fr_akas

,key_0,titleId,title,types,region,titleType
366,tt0000591,tt0000591,L'enfant prodigue,imdbDisplay,FR,movie
392,tt0000886,tt0000886,Hamlet,imdbDisplay,FR,movie
410,tt0001101,tt0001101,La Clémence d'Abraham Lincoln,imdbDisplay,FR,movie
411,tt0001109,tt0001109,L'aluminite,imdbDisplay,FR,movie
413,tt0001122,tt0001122,L'auberge rouge,imdbDisplay,FR,movie
...,...,...,...,...,...,...
4044164,tt6867320,tt6867320,Les cercles du vicieux,NaN,FR,movie
4150053,tt7443988,tt7443988,Global gâchis,NaN,FR,movie
4186078,tt7662854,tt7662854,Le Viceconsul,NaN,FR,movie
4322046,tt8309482,tt8309482,"DESTIN COMMUN, Hommage à 3 précurseurs du 7ème...",NaN,FR,movie


In [50]:
fr_akas["titleId"].value_counts()

titleId
tt0000591     1
tt19363554    1
tt1935956     1
tt1935947     1
tt1935940     1
             ..
tt0135470     1
tt0135459     1
tt0135454     1
tt0135397     1
tt9234412     1
Name: count, Length: 85289, dtype: int64

In [51]:
# Merging (INNER) movies and akas to get only the translated movies
movies_frt = df_movie.merge(fr_akas["title"], how = "inner", left_on = df_movie["tconst"], right_on = fr_akas["titleId"])

In [52]:
movies_frt

,key_0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,title
0,tt0000591,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.000,1907.000,NaN,90,Drama,L'enfant prodigue
1,tt0000886,tt0000886,movie,"Hamlet, Prince of Denmark",Hamlet,0.000,1910.000,NaN,NaN,Drama,Hamlet
2,tt0001101,tt0001101,movie,Abraham Lincoln's Clemency,Abraham Lincoln's Clemency,0.000,1910.000,NaN,NaN,NaN,La Clémence d'Abraham Lincoln
3,tt0001109,tt0001109,movie,L'aluminite,L'aluminite,0.000,1910.000,NaN,NaN,NaN,L'aluminite
4,tt0001122,tt0001122,movie,The Red Inn,L'auberge rouge,0.000,1910.000,NaN,NaN,NaN,L'auberge rouge
...,...,...,...,...,...,...,...,...,...,...,...
83916,tt9913936,tt9913936,movie,Paradise,Paradise,0.000,2019.000,NaN,135,"Crime,Drama",Le paradis de Diego
83917,tt9914164,tt9914164,movie,Une vie de château,Une vie de château,0.000,2019.000,NaN,55,Documentary,Une vie de château
83918,tt9915790,tt9915790,movie,Bobbyr Bondhura,Bobbyr Bondhura,0.000,2019.000,NaN,106,Family,Bobbyr Bondhura
83919,tt9916162,tt9916162,movie,Making of 'La Virgen de los sicarios',Making of 'La Virgen de los sicarios',0.000,1999.000,NaN,45,Documentary,Making of 'La Virgen de los sicarios'


In [53]:
title_rate

,tconst,averageRating,numVotes
0,tt0000001,5.700,2041
1,tt0000002,5.700,273
2,tt0000003,6.500,1998
3,tt0000004,5.400,178
4,tt0000005,6.200,2758
...,...,...,...
1425216,tt9916730,7.000,12
1425217,tt9916766,7.100,23
1425218,tt9916778,7.200,36
1425219,tt9916840,7.000,9


In [54]:
movies_frt.drop(columns = "key_0", inplace = True)

In [55]:
# Merging (LEFT) fr_movies with ratings
movies_frt = movies_frt.merge(title_rate[["averageRating", "numVotes"]], how = "left", left_on = movies_frt["tconst"], right_on = title_rate["tconst"])

In [56]:
movies_frt.drop(columns = "key_0", inplace = True)

In [57]:
# Final selection of movies before dropping columns

movie_final = movies_frt.merge(TMdb, how = "left", left_on = movies_frt["tconst"], right_on = TMdb["imdb_id"])

In [58]:
display(movie_final.info(),
        movie_final.describe(),
        movie_final.sample(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83921 entries, 0 to 83920
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   key_0                         83921 non-null  object 
 1   tconst                        83921 non-null  object 
 2   titleType                     83921 non-null  object 
 3   primaryTitle                  83921 non-null  object 
 4   originalTitle                 83921 non-null  object 
 5   isAdult                       83921 non-null  float64
 6   startYear                     82369 non-null  float64
 7   endYear                       0 non-null      float64
 8   runtimeMinutes                77273 non-null  object 
 9   genres_x                      80261 non-null  object 
 10  title_x                       83921 non-null  object 
 11  averageRating                 67794 non-null  float64
 12  numVotes                      67794 non-null  float64
 13  a

None

,isAdult,startYear,endYear,averageRating,numVotes,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,83921.000,82369.000,0.000,67794.000,67794.000,60640.000,60640.000,60640.000,60640.000,60640.000,60640.000,60640.000
mean,0.002,1987.468,NaN,6.112,15429.025,4104837.435,256218.334,7.448,10967863.607,94.777,5.416,299.296
std,0.044,31.050,NaN,1.117,75769.727,17980564.376,275929.635,46.959,66933568.080,184.508,2.083,1299.790
min,0.000,1903.000,NaN,1.000,5.000,0.000,2.000,0.600,0.000,0.000,0.000,0.000
25%,0.000,1965.000,NaN,5.500,113.000,0.000,43503.750,1.220,0.000,86.000,5.000,4.000
50%,0.000,1997.000,NaN,6.200,624.000,0.000,135981.000,2.686,0.000,95.000,6.000,16.000
75%,0.000,2015.000,NaN,6.900,3652.000,0.000,407967.000,7.404,0.000,105.000,6.700,89.000
max,1.000,2031.000,NaN,10.000,2881964.000,579330426.000,1122335.000,5089.969,2923706026.000,43200.000,10.000,33630.000


,key_0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_x,title_x,averageRating,numVotes,adult,backdrop_path,budget,genres_y,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,video,vote_average,vote_count,production_companies_name,production_companies_country
69196,tt28753771,tt28753771,movie,Mixing,Métissages,0.000,2023.000,NaN,70,Documentary,Métissages,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5409,tt0029870,tt0029870,movie,Angels with Dirty Faces,Angels with Dirty Faces,0.000,1938.000,NaN,97,"Crime,Drama,Film-Noir",Les anges aux figures sales,7.900,23172.000,False,/bb36aEZwEvK2L01mgWQuC506CZg.jpg,0.000,"['Crime', 'Drama', 'Thriller']",NaN,13696.000,tt0029870,en,Angels with Dirty Faces,Childhood chums Rocky Sullivan and Jerry Conne...,8.198,/k23E4UAcow8eczLRmVCMdukL4Mx.jpg,['US'],1938-11-26,0.000,97.000,['en'],Released,The saga of America’s dirty faced kids... and ...,Angels with Dirty Faces,False,7.645,262.000,['Warner Bros. Pictures'],['US']
21623,tt0080630,tt0080630,movie,Dirty Tricks,Dirty Tricks,0.000,1980.000,NaN,95,Comedy,"Accroche-toi, j'arrive",4.700,156.000,False,NaN,0.000,['Comedy'],NaN,262842.000,tt0080630,en,Dirty Tricks,A Harvard University student finds a strange l...,1.526,/oSuJowsD37IAd7ceCcviuX656m5.jpg,[],1981-03-06,0.000,95.000,['en'],Released,Everyone here is crazy...everyone else is crac...,Dirty Tricks,False,0.000,0.000,[],NaN
44487,tt0400525,tt0400525,movie,The Ice Harvest,The Ice Harvest,0.000,2005.000,NaN,92,"Comedy,Crime,Drama",Faux amis,6.200,24990.000,False,/kkPhYPtJnUvEwJycOtxYFAAStH9.jpg,0.000,"['Action', 'Adventure', 'Comedy', 'Crime', 'Dr...",NaN,9005.000,tt0400525,en,The Ice Harvest,"A shady lawyer attempts a Christmas Eve crime,...",8.788,/8N2X2u6nPoPEwRLIZbKacJw1xrC.jpg,['US'],2005-11-23,0.000,88.000,['en'],Released,Thick Thieves. Thin Ice.,The Ice Harvest,False,5.970,252.000,"['Focus Features', 'Bona Fide Productions']","['US', 'US']"
1437,tt0012989,tt0012989,movie,La bête traquée,La bête traquée,0.000,1923.000,NaN,NaN,NaN,La bête traquée,NaN,NaN,False,NaN,0.000,[],NaN,555923.000,tt0012989,xx,La bête traquée,NaN,0.600,NaN,[],1923-03-09,0.000,89.000,[],Released,NaN,The Hunted Beast,False,0.000,0.000,[],[]
9182,tt0043989,tt0043989,movie,Sailor Beware,Sailor Beware,0.000,1952.000,NaN,108,"Comedy,Musical,Romance",Escale à Honululu,6.200,1385.000,False,/ojQ90CIU49QwnEwY9LWpYDaCyjd.jpg,0.000,['Comedy'],NaN,39015.000,tt0043989,en,Sailor Beware,"Meeting in a navy recruiting line, Al Crowther...",5.176,/fksEN0tiQMbWYwh9Z1OH4fL3yHp.jpg,['US'],1952-02-08,0.000,108.000,['en'],Released,They go down to the sea in ships… and come up ...,Sailor Beware,False,6.160,25.000,"['Paramount', 'Wallis-Hazen Inc.']",NaN
18771,tt0070350,tt0070350,movie,The Mack,The Mack,0.000,1973.000,NaN,110,"Crime,Drama",Le mac,6.600,3291.000,False,/3hXaX4dL8U9KLoQaON0mkVM9gwS.jpg,0.000,['Drama'],NaN,29515.000,tt0070350,en,The Mack,Goldie returns from five years at the state pe...,4.557,/xcS9bxWGFzC3cA7qAOjcIjKajSf.jpg,['US'],1973-04-04,0.000,110.000,['en'],Released,There's a new kind of hero on the streets!,The Mack,False,6.500,47.000,['Harbor Productions'],NaN
72389,tt3672616,tt3672616,movie,Fulboy,Fulboy,0.000,2015.000,NaN,82,"Biography,Documentary,Sport",Les Garçons du stade,4.900,202.000,False,/9Vdhkry2tbLni7pNnOjWdDSfNmx.jpg,0.000,['Documentary'],NaN,332600.000,tt3672616,es,Fulboy,"On the verge of achieving his dream career, To...",1.325,/bIIs0z4yYC1IHoC7zB8WQib2qPQ.jpg,['AR'],2015-03-21,0.000,82.000,['es'],Released,NaN,Fulboy,False,4.300,12.000,"['TLA Releasing', 'Cinemilagroso']",NaN
1627,tt0013767,tt0013767,movie,Wildcat Jordan,Wildcat Jordan,0.000,1922.000,NaN,NaN,"Comedy,Drama",Diavolo enragé,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [59]:
# Which TMdb genre for NA genre in IMDB
movie_final[movie_final["genres_x"].isna()]["genres_y"].unique()

array(['[]', nan, "['Drama', 'History']", "['Drama']",
       "['Science Fiction']", "['Mystery', 'Crime']", "['Comedy']",
       "['Adventure']", "['Drama', 'Comedy']", "['Crime', 'Drama']",
       "['Comedy', 'Drama']", "['Drama', 'Action', 'Adventure']",
       "['Music', 'Comedy']", "['Adventure', 'Drama']",
       "['Romance', 'Comedy']", "['Romance', 'Drama']",
       "['Drama', 'War']", "['Comedy', 'Music']", "['Drama', 'Music']",
       "['Music', 'Romance', 'Drama']", "['Family', 'Music']",
       "['Music']", "['Crime']", "['Romance']", "['Drama', 'Family']",
       "['Documentary']", "['Comedy', 'Drama', 'Romance']",
       "['Drama', 'Documentary']", "['Thriller']", "['Crime', 'Comedy']",
       "['Thriller', 'Drama']", "['Family', 'Drama']",
       "['Comedy', 'Romance']", "['Drama', 'Romance']",
       "['Action', 'Comedy']", "['Action', 'Adventure', 'History']",
       "['History']", "['Crime', 'Action', 'Drama']",
       "['Crime', 'Action']", "['Drama', 'Drama']",
    

In [60]:
to_add = movie_final[(movie_final["genres_x"].isna()) & (movie_final["genres_y"].notna()) & (movie_final["genres_y"] != "[]")]

In [61]:
to_add

,key_0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_x,title_x,averageRating,numVotes,adult,backdrop_path,budget,genres_y,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,video,vote_average,vote_count,production_companies_name,production_companies_country
18,tt0002375,tt0002375,movie,La mort du duc d'Enghien,La mort du duc d'Enghien,0.000,1912.000,NaN,NaN,NaN,La mort du duc d'Enghien,5.700,12.000,False,NaN,0.000,"['Drama', 'History']",NaN,285335.000,tt0002375,en,La Mort du duc d'Enghien,Directed by Albert Capellani.,0.823,/1D9MCUxnl4nzFVlJNsppP6Sdk94.jpg,['FR'],1912-01-02,0.000,15.000,[],Released,NaN,The Death of the Duke of Enghien,False,0.000,1.000,['Pathé'],NaN
58,tt0003960,tt0003960,movie,Loyalty,La fille de Delft,0.000,1914.000,NaN,68,NaN,La tulipe d'or,6.100,33.000,False,NaN,0.000,['Drama'],NaN,321824.000,tt0003960,fr,La fille de Delft,"Kaatje, the daughter of the miller Schoonejans...",1.400,NaN,['BE'],1914-04-23,0.000,71.000,['fr'],Released,NaN,Loyalty,False,6.500,1.000,['Pathé Frères'],NaN
117,tt0005813,tt0005813,movie,Nikolay Stavrogin,Nikolay Stavrogin,0.000,1915.000,NaN,NaN,NaN,Nicolas Stavroguine,NaN,NaN,False,NaN,0.000,['Drama'],NaN,878872.000,tt0005813,ru,Николай Ставрогин,NaN,0.600,NaN,['RU'],1915-09-01,0.000,0.000,[],Released,NaN,Nikolay Stavrogin,False,0.000,0.000,[],[]
135,tt0006233,tt0006233,movie,Volonté,Volonté,0.000,1917.000,NaN,NaN,NaN,Volonté,NaN,NaN,False,NaN,0.000,['Drama'],NaN,282258.000,tt0006233,en,Volonté,"The story takes place in the Parisian ""High So...",0.600,NaN,['FR'],1917-04-13,0.000,0.000,['fr'],Released,NaN,Will,False,0.000,0.000,[],NaN
183,tt0006713,tt0006713,movie,Les gaz mortels,Les gaz mortels,0.000,1916.000,NaN,71,NaN,Le brouillard sur la ville,6.000,48.000,False,NaN,0.000,['Science Fiction'],NaN,607385.000,tt0006713,fr,Les gaz mortels,"Hopson, a prestigious scientist, studies the e...",1.200,/wgO8saob0oVBtzUzi1WYcuJmKGO.jpg,['FR'],1916-09-01,0.000,71.000,['fr'],Released,NaN,Les gaz mortels,False,5.000,1.000,"[""Le Film d'Art""]",['FR']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80963,tt7999612,tt7999612,movie,Mescaline,Mescaline,0.000,2018.000,NaN,45,NaN,Mescaline,6.400,14.000,False,NaN,0.000,['Drama'],NaN,506605.000,tt7999612,fr,Mescaline,Under the influence of a hallucinogenic cactus...,1.379,/xskj9TTbEykHGg1XXfNGBjnBqop.jpg,['FR'],2018-02-03,0.000,45.000,['fr'],Released,NaN,Mescaline,False,7.500,1.000,"['24images', 'Les films du Bélier']","['FR', 'FR']"
82161,tt8699536,tt8699536,movie,Monsieur Constant,Monsieur Constant,0.000,2023.000,NaN,108,NaN,Monsieur Constant,4.600,5.000,False,NaN,0.000,"['Comedy', 'Drama']",NaN,1064998.000,tt8699536,fr,Monsieur Constant,"On the island of the monks in Brittany, Consta...",0.680,/rI9jYMTYjR5AVMxQcPshTtsf35A.jpg,[],2023-02-08,0.000,0.000,"['fr', 'ru']",Released,NaN,Monsieur Constant,False,0.000,0.000,[],[]
82167,tt8705634,tt8705634,movie,Facing Krampus,Klausentreiben,0.000,2021.000,NaN,45,NaN,Gare au Krampus,NaN,NaN,False,NaN,30000.000,"['Thriller', 'Horror']",https://www.filmschule.de/en/projects-films/kl...,890967.000,tt8705634,de,Klausentreiben,During her internship in a remote clinic in th...,0.645,/mXjcpwjZFWKZLQnyzSB62xzXa8W.jpg,['DE'],2021-05-15,0.000,45.000,['de'],Released,NaN,Facing Krampus,False,10.000,1.000,"['ifs internationale filmschule köln gmbh', 'F...","['DE', 'DE']"
82369,tt8832044,tt8832044,movie,Régénération,Régénération,0.000,2018.000,NaN,86,NaN,Régénération,NaN,NaN,False,/kBY0qDK4XsL9OSziAimAeRtV8UP.jpg,0.000,['Documentary'],NaN,591053.000,tt8832044,fr,Régénération,NaN,0.885,/o4oLL5g6fE7yNORyo5jsfFizzk4.jpg,['FR'],2018-11-21,0.000,0.000,['fr'],Released,NaN,Regeneration,False,10.000,1.000,['DHR'],['']


In [62]:
movie_final.drop(movie_final[movie_final["genres_x"].isna()].index, inplace = True)

In [63]:
movie_final

,key_0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_x,title_x,averageRating,numVotes,adult,backdrop_path,budget,genres_y,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,video,vote_average,vote_count,production_companies_name,production_companies_country
0,tt0000591,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.000,1907.000,NaN,90,Drama,L'enfant prodigue,5.400,24.000,False,NaN,0.000,['Drama'],NaN,396922.000,tt0000591,fr,L'enfant prodigue,The first feature-length motion picture produc...,0.600,NaN,['FR'],1907-06-19,0.000,90.000,['xx'],Released,NaN,The Prodigal Son,False,0.000,0.000,['Pathé Frères'],['FR']
1,tt0000886,tt0000886,movie,"Hamlet, Prince of Denmark",Hamlet,0.000,1910.000,NaN,NaN,Drama,Hamlet,4.600,41.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,tt0001175,tt0001175,movie,Camille,La dame aux camélias,0.000,1912.000,NaN,NaN,"Drama,Romance",La dame aux camélias,5.200,45.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,tt0001614,tt0001614,movie,The Four Devils,De fire djævle,0.000,1911.000,NaN,60,Drama,Les Quatre Diables,6.400,42.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,tt0001790,tt0001790,movie,"Les Misérables, Part 1: Jean Valjean",Les misérables - Époque 1: Jean Valjean,0.000,1913.000,NaN,60,Drama,Les misérables - Époque 1: Jean Valjean,6.000,52.000,False,NaN,0.000,['History'],NaN,282219.000,tt0001790,en,Les misérables - Époque 1: Jean Valjean,The story begins with Jean Valjean as a humble...,0.874,NaN,['FR'],1913-01-01,0.000,60.000,['fr'],Released,NaN,Les Misérables - Part 1: Jean Valjean,False,1.000,1.000,['Pathé Frères'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83916,tt9913936,tt9913936,movie,Paradise,Paradise,0.000,2019.000,NaN,135,"Crime,Drama",Le paradis de Diego,7.400,61.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83917,tt9914164,tt9914164,movie,Une vie de château,Une vie de château,0.000,2019.000,NaN,55,Documentary,Une vie de château,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83918,tt9915790,tt9915790,movie,Bobbyr Bondhura,Bobbyr Bondhura,0.000,2019.000,NaN,106,Family,Bobbyr Bondhura,6.900,43.000,False,/9Zqup9L9zGMyDijAeKFkelcknp.jpg,0.000,['Drama'],NaN,591918.000,tt9915790,bn,Bobbyr Bondhura,Little Bobby goes missing after the separation...,0.600,/6trvIgsbe5rBlL1GoR6rJnNA3tv.jpg,['IN'],2019-03-08,0.000,106.000,[],Released,NaN,Bobbyr Bondhura,False,0.000,0.000,[],[]
83919,tt9916162,tt9916162,movie,Making of 'La Virgen de los sicarios',Making of 'La Virgen de los sicarios',0.000,1999.000,NaN,45,Documentary,Making of 'La Virgen de los sicarios',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
movie_final = pd.concat([movie_final, to_add])

In [65]:
movie_final

,key_0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres_x,title_x,averageRating,numVotes,adult,backdrop_path,budget,genres_y,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,video,vote_average,vote_count,production_companies_name,production_companies_country
0,tt0000591,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.000,1907.000,NaN,90,Drama,L'enfant prodigue,5.400,24.000,False,NaN,0.000,['Drama'],NaN,396922.000,tt0000591,fr,L'enfant prodigue,The first feature-length motion picture produc...,0.600,NaN,['FR'],1907-06-19,0.000,90.000,['xx'],Released,NaN,The Prodigal Son,False,0.000,0.000,['Pathé Frères'],['FR']
1,tt0000886,tt0000886,movie,"Hamlet, Prince of Denmark",Hamlet,0.000,1910.000,NaN,NaN,Drama,Hamlet,4.600,41.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,tt0001175,tt0001175,movie,Camille,La dame aux camélias,0.000,1912.000,NaN,NaN,"Drama,Romance",La dame aux camélias,5.200,45.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,tt0001614,tt0001614,movie,The Four Devils,De fire djævle,0.000,1911.000,NaN,60,Drama,Les Quatre Diables,6.400,42.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,tt0001790,tt0001790,movie,"Les Misérables, Part 1: Jean Valjean",Les misérables - Époque 1: Jean Valjean,0.000,1913.000,NaN,60,Drama,Les misérables - Époque 1: Jean Valjean,6.000,52.000,False,NaN,0.000,['History'],NaN,282219.000,tt0001790,en,Les misérables - Époque 1: Jean Valjean,The story begins with Jean Valjean as a humble...,0.874,NaN,['FR'],1913-01-01,0.000,60.000,['fr'],Released,NaN,Les Misérables - Part 1: Jean Valjean,False,1.000,1.000,['Pathé Frères'],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80963,tt7999612,tt7999612,movie,Mescaline,Mescaline,0.000,2018.000,NaN,45,NaN,Mescaline,6.400,14.000,False,NaN,0.000,['Drama'],NaN,506605.000,tt7999612,fr,Mescaline,Under the influence of a hallucinogenic cactus...,1.379,/xskj9TTbEykHGg1XXfNGBjnBqop.jpg,['FR'],2018-02-03,0.000,45.000,['fr'],Released,NaN,Mescaline,False,7.500,1.000,"['24images', 'Les films du Bélier']","['FR', 'FR']"
82161,tt8699536,tt8699536,movie,Monsieur Constant,Monsieur Constant,0.000,2023.000,NaN,108,NaN,Monsieur Constant,4.600,5.000,False,NaN,0.000,"['Comedy', 'Drama']",NaN,1064998.000,tt8699536,fr,Monsieur Constant,"On the island of the monks in Brittany, Consta...",0.680,/rI9jYMTYjR5AVMxQcPshTtsf35A.jpg,[],2023-02-08,0.000,0.000,"['fr', 'ru']",Released,NaN,Monsieur Constant,False,0.000,0.000,[],[]
82167,tt8705634,tt8705634,movie,Facing Krampus,Klausentreiben,0.000,2021.000,NaN,45,NaN,Gare au Krampus,NaN,NaN,False,NaN,30000.000,"['Thriller', 'Horror']",https://www.filmschule.de/en/projects-films/kl...,890967.000,tt8705634,de,Klausentreiben,During her internship in a remote clinic in th...,0.645,/mXjcpwjZFWKZLQnyzSB62xzXa8W.jpg,['DE'],2021-05-15,0.000,45.000,['de'],Released,NaN,Facing Krampus,False,10.000,1.000,"['ifs internationale filmschule köln gmbh', 'F...","['DE', 'DE']"
82369,tt8832044,tt8832044,movie,Régénération,Régénération,0.000,2018.000,NaN,86,NaN,Régénération,NaN,NaN,False,/kBY0qDK4XsL9OSziAimAeRtV8UP.jpg,0.000,['Documentary'],NaN,591053.000,tt8832044,fr,Régénération,NaN,0.885,/o4oLL5g6fE7yNORyo5jsfFizzk4.jpg,['FR'],2018-11-21,0.000,0.000,['fr'],Released,NaN,Regeneration,False,10.000,1.000,['DHR'],['']


In [66]:
list_of_movie = movie_final["tconst"].unique()

In [68]:
pd.DataFrame(list_of_movie).to_csv("list_of_movie.csv")

From this list of movie, we filter the names thanks to title principals

In [69]:
link_prin = r"C:\Users\User\Wild_Code\Projects\Project_2\DB\title.principals.tsv.gz"
title_prin = pd.read_csv(link_prin, sep = "\t", na_values = r"\N")

In [70]:
title_prin[title_prin["tconst"].isin(list_of_movie)]

,tconst,ordering,nconst,category,job,characters
1421,tt0000591,1,nm0906197,actor,NaN,NaN
1422,tt0000591,2,nm0332182,actor,NaN,NaN
1423,tt0000591,3,nm1323543,actress,NaN,NaN
1424,tt0000591,4,nm1759558,actress,NaN,NaN
1425,tt0000591,5,nm0141150,director,NaN,NaN
...,...,...,...,...,...,...
85085818,tt9916362,20,nm0284765,editor,NaN,NaN
85085819,tt9916362,21,nm2857592,casting_director,NaN,NaN
85085820,tt9916362,22,nm1486091,casting_director,casting_director,NaN
85085821,tt9916362,23,nm7266158,casting_director,NaN,NaN
